In [1]:
import create_vocab
import data_to_tensors
import model_implementation
from train_class import TrainingModule

import torch
import torch.nn as nn
import torch.nn.functional as F
import random 

from torch.utils.data import DataLoader

RuntimeError: Failed to import transformers.models.bert because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
No module named 'tensorflow.python.keras.engine.keras_tensor'

In [2]:
data_folder = 'CODES_emb'
dataset_name = 'CODES_emb'
dict_path = 'data/'+ data_folder + '/' + dataset_name + '.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

In [3]:
path_for_train = 'data/'+ data_folder + '/' + dataset_name + '.train.c2v'
train_dataset = data_to_tensors.TextDataset(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_val = 'data/'+ data_folder + '/' + dataset_name + '.val.c2v'
val_dataset = data_to_tensors.TextDataset(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_test = 'data/'+ data_folder + '/' + dataset_name + '.test.c2v'
test_dataset = data_to_tensors.TextDataset(path_for_test, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Train

In [4]:
SEED = 1337
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
bert = False

In [6]:
# In case of bert
bert_params = dict()
bert_params['num_attention_heads'] = 1
bert_params['num_transformer_layers'] = 1
bert_params['intermediate_size'] = 32

if bert:
    model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                             paths_vocab_size = len(path2idx), 
                             labels_num = len(target2idx), bert=bert, bert_params=bert_params)

In [7]:
# In case if not bert
if not bert:
    model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                                 paths_vocab_size = len(path2idx), 
                                 labels_num = len(target2idx))

In [8]:
N_EPOCHS = 40
LR = 3e-3
WD = 0.8e-5

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
criterion = nn.CrossEntropyLoss()

train_class = TrainingModule(model, optimizer, criterion, train_loader, val_loader, test_loader, N_EPOCHS, idx2target)
list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1 = train_class.train(bert, dataset_name)


Epoch 1: train loss - 1.50903, validation loss - 1.55745
	 Validation: precision - 0.35484, recall - 0.35484, f1_score - 0.35484
Elapsed time: 3.395
----------------------------------------------------------------------
Epoch 2: train loss - 1.14902, validation loss - 1.34985
	 Validation: precision - 0.43548, recall - 0.43548, f1_score - 0.43548
Elapsed time: 4.398
----------------------------------------------------------------------
Epoch 3: train loss - 1.09201, validation loss - 1.29309
	 Validation: precision - 0.40323, recall - 0.40323, f1_score - 0.40323
Elapsed time: 8.789
----------------------------------------------------------------------
Epoch 4: train loss - 1.07962, validation loss - 1.29608
	 Validation: precision - 0.37097, recall - 0.37097, f1_score - 0.37097
Elapsed time: 11.458
----------------------------------------------------------------------
Epoch 5: train loss - 1.05642, validation loss - 1.28502
	 Validation: precision - 0.41935, recall - 0.41935, f1_score 

In [11]:
if bert == True:
  # state_dict = torch.load('best_model.pth')
  state_dict = torch.load('./' + dataset_name + '_bert_model.pth')
else:
  state_dict = torch.load( './' + dataset_name + '_article_model.pth')

model.load_state_dict(state_dict)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(DEVICE)

In [12]:
import pandas as pd
from IPython.display import display

d = {'Original names': [], 'Predicted names': []}

for start, path, end, label in iter(test_loader):
    code, y_pred = model(start.to(DEVICE), path.to(DEVICE), end.to(DEVICE))
    y_pred = F.softmax(y_pred)
    y_pred = torch.argmax(y_pred, dim = 1)
    
    for i, j in zip(label, y_pred):
      d['Original names'].append(idx2target[i.item()])
      d['Predicted names'].append(idx2target[j.item()])
    break

df = pd.DataFrame(data=d)
display(df,)

/Users/unaissiddiqui/opt/anaconda3/envs/Fyp/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


,Original names,Predicted names
0,<unk>,7
1,7,7
2,2,10
3,7,7
4,5,10
...,...,...
57,7,7
58,10,7
59,7,7
60,7,10
